In [5]:
import yaml
import settings
import pickle
import pandas as pd

# Load Raw Data
- set `do_reload_from_bq` to True if want to read Data from GBQ else data will be loaded from local pickle file
- if `do_reload_from_bq` & `store_dataset` both are True then data from GBQ will be stored in local pickle file

In [6]:
class TuringDataset:

    def __init__(self,filetype='pickle', do_reload_from_bq=False, store_dataset=False):

        if do_reload_from_bq is True:
            query_config_path = settings.query_config_path

            self.query_dict = {}
            self.dataset = {}
            with open(query_config_path, "r") as query_config_file:
                self.query_dict = yaml.safe_load(query_config_file)

            dataset = self.load_dataset_using_query_file()
        else:
            dataset = pd.read_pickle(settings.raw_dataset_path)
            
        self.dataset = dataset
        
        if store_dataset is True and do_reload_from_bq is True:
            self.dump_dataset_into_file(self.dataset, filepath=settings.raw_dataset_path,
                                          filetype=filetype)
                   
    def load_dataset_using_query_file(self):
        dataset = {}
        for dataset_name, query_data in self.query_dict.items():
            query = query_data['query']
            data_df = pd.read_gbq(query)
            dataset[dataset_name] = data_df
        return dataset
    
    def dump_dataset_into_file(self, dataset, filepath='../data', filetype='pkl'):
        
        with open(filepath, 'wb') as handle:
            pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
             
    def get_dataset_df(self):
        return self.dataset


## Raw data


In [7]:
# Instance of TuringDataset class
TuringDataset_instance = TuringDataset(filetype='pickle', do_reload_from_bq=False, store_dataset=True)
dataset = TuringDataset_instance.get_dataset_df()
dataset.keys()

dict_keys(['projects', 'jobs'])

In [8]:
projects = dataset['projects']
jobs = dataset['jobs']
del dataset

## Preprocessing

In [ ]:
# Some preprocessing and then storing all data frames in a dictionary

In [9]:
processd_dataset = {}

processd_dataset['X'] = projects
processd_dataset['Y'] = jobs

In [11]:
with open(settings.processed_dataset_path, 'wb') as handle:
        pickle.dump(processd_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)